In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import scipy.interpolate as interpolate
import matplotlib.colors as mcolors
import matplotlib
import itertools as it
from collections.abc import Iterable

In [ ]:
df_all=pd.read_hdf('/mnt/DATA/processed_swarm_data_noisepos0.5_noisephi0.1_N225_forAnna.h5')
convert_dict={'N':int,'number_connected_components':int}
df_all=df_all.astype(convert_dict)
df_all['aspect']=df_all['aspect'].round(decimals=1)
df_all['alpha']=df_all['alpha'].round(decimals=2)

df_all['threshold']=df_all['threshold'].round(decimals=4)
df_all['noisephi']=df_all['noisephi'].round(decimals=4)

In [ ]:
def dataframe_filter(filters=None,setup='grid',networktype='visual'):
    # filter the dataframe to only keep the required network type, aspect ratio,
    # setup and N (number of individuals) and further limitations added by the filters dictionary
    
    filterquery=''
    filtertext=''
    if filters is not None:
        for k in filters.keys():
            if isinstance(filters[k], Iterable) and not isinstance(filters[k],str):
                filterquery+=k+' <= %1.5f & '%filters[k][1]+k+' >=%1.5f &'%filters[k][0]
                filtertext+=k+'$\in$ [%1.3f,%1.3f]\n'%(filters[k][0],filters[k][1])
            elif isinstance(filters[k], float) or isinstance(filters[k],int):
                filterquery+=k+' == %1.5f &'%(filters[k])
                filtertext+=k+' = %1.3f\n'%filters[k]
            else:
                print('no idea what it is that you want to filter for: ',k,filters[k])
    filterquery+='networktype=="%s" & setup=="%s"'%(networktype,setup)
    filtertext+='networktype ="%s"\nsetup = "%s"'%(networktype,setup)
    return [filterquery,filtertext]

In [ ]:
def plot_line(df_all,xvar,yvar,setup='grid',networktype='visual',
              xbins=None,nr_xbins=100,ci=True,color='k',ax=None,xlabel=None,ylabel=None,
                ls='-',alpha=1,filters=None,annotateparams=True,label=None,
              annoteplace=(1.8,-0.1),lw=1,min_stats=0):
    # filter the data we need
    filterquery,filtertext=dataframe_filter(filters=filters,setup=setup,networktype=networktype)
    df_tmp=df_all.query(filterquery)[[xvar,yvar]]

    if ax is None:
        fig,ax=plt.subplots(1)
    if xlabel is None:
            xlabel=xvar
    if ylabel is None:
            ylabel=yvar
    if xbins is None:
        xmax=np.nanmax(df_tmp[xvar])
        xmin=np.nanmin(df_tmp[xvar])
        xbins=np.linspace(xmin,xmax,nr_xbins+1,endpoint=True)
        
    # bin the data on the x-axis and average bins
    xbinlabels=xbins[:-1]+0.5*np.diff(xbins)
    df_tmp[xvar+'_binned']=pd.cut(df_tmp[xvar],bins=xbins,labels=xbinlabels)
    df_tmp=df_tmp.dropna(axis=0)
    df_tmp.drop(columns=xvar,inplace=True)
    tmp = df_tmp.groupby(xvar+'_binned').agg(['mean','count','std']).swaplevel(0,1,axis=1)
    df_mean=tmp['mean'].where(tmp['count']>=min_stats).reset_index().dropna()
    df_std=tmp['std'].where(tmp['count']>=min_stats).reset_index().dropna()
    
    # plot the averaged data
    xm=np.array(df_mean[xvar+'_binned'])
    ym=np.array(df_mean[yvar])
    ys=np.array(df_std[yvar])
    if label is None:
        label=networktype

    ax.plot(xm,ym,color=color,alpha=alpha,label=label,ls=ls,lw=lw)
    ax.set_xlabel(xlabel)
    #plot confidence intervals of one standard deviation if required
    if ci:
        ax.fill_between(xm,ym-ys,y2=ym+ys,
                   color=color,alpha=0.1,label='_no_legend')

    ax.set_ylabel(ylabel)
    if annotateparams:
        ax.annotate(filtertext,annoteplace,xycoords='axes fraction', horizontalalignment='right')     


In [ ]:
#check which alphas are in the dataframe
df_all.alpha.unique()

In [ ]:
N=225

axlabel={'avg_linklength':'avg. link length','rel_linklength':'avg. rel. link length',
         'avg_indegree':'avg. indegree','avg_outdegree':'avg. outdegree',
        'avg_clustering':'avg. clust. coeff.',
        'avg_shortest_path':'avg. shortest path','density':'density [BL$^{-2}$]',
         'new_density':'density [BL$^{-2}$]','avg_instrength':'avg. in-strength'}
main_colors={'visual':'indigo','metric':'coral','topological':'seagreen'}

fig,ax=plt.subplots(1)
ax.set_xscale('log')

thresholds=[0., 0.01 , 0.1 ]
norm=plt.Normalize(vmin=np.amin(thresholds), vmax=np.amax(thresholds))
cmap=plt.cm.viridis
yvar='avg_indegree'


for i,thresh in enumerate(thresholds):
    c=cmap(norm(thresh))
    filters={'polarization':[0,0.3],'threshold':thresh,'noisepos':0.5,'N':N,'aspect':0.3,'alpha':0,
             'number_connected_components':1}
    xbins=np.logspace(np.log10(0.005),np.log10(2.2),50)
    plot_line(df_all,'density',yvar,
            networktype='visual',xbins=xbins,ci=False,color=c,annotateparams=False,
              alpha=None,min_stats=5,ax=ax,filters=filters,
              label='threshold=%1.3f'%thresh)

ax.set_ylabel(axlabel[yvar])
ax.set_xlabel('density [BL^{-2}]')

sm = plt.cm.ScalarMappable(cmap=cmap,norm=norm)
sm._A = [] # this might not be necessary if your matplotlib is newer than mine

colorbar=plt.colorbar(sm,ticks=thresholds)
colorbar.set_label('visual threshold')

x_minor = matplotlib.ticker.LogLocator(base = 10.0, subs = np.arange(1.0, 10.0) * 0.1, numticks = 10)
ax.xaxis.set_minor_locator(x_minor)
ax.xaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
fig.set_size_inches(2.3622*.65,1.25)

In [ ]:
# here you could make a similar plot but instead of varying the visual threshold you would vary the alpha
# and keep a fixed threshold

# Also, you should look at the avg_instrength (which is the weighted indegree) instead of avg_indegree
# As a sanity check you can plot the avg_indegree and it should always look like one of the above lines 
# (depending on your visualthreshold value)

#You can either write your own plotting function or use the one above.